In [1]:
import os
import sys
from pathlib import Path
django_project_dir = Path(".")
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "newsilkroad.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

[raven.contrib.django.client.DjangoClient]DEBUG 13:25:28 Configuring Raven for host: None
[raven.contrib.django.client.DjangoClient]INFO 13:25:28 Raven is not configured (logging is disabled). Please see the documentation for more information.


In [2]:
from writings.models import Entry, EntryCollection
import csv

In [6]:
def _get_categories(obj, *args):
    if obj.categories:
        return ",".join([x.name for x in obj.categories.all()])
    return ""

def _get_tags(obj, *args):
    if obj.tags:
        return ",".join([x.name for x in obj.tags.all()])
    return ""

def _get_related_entries(obj, *args):
    if obj.related_entries:
        return ",".join([x.title for x in obj.related_entries.all()])
    return ""

def _get_filer_info(obj, *args):
    value = ''
    try:
        if args[0] == "featured_image_id":
            value = obj.featured_image.original_filename
        else:
            value = obj.sponsored_logo.original_filename
    except AttributeError as e:
        pass
    return value

In [7]:
entry_fields =  {
    'id': '',
    'created_at': '',
    'updated_at': '',
    'published': '',
    'title': '',
    'subtitle': '',
    'slug': '',
    'author': '',
    'content': '',
    'content_rendered': '',
    'description': '',
    'description_rendered': '',
    'share_text': '',
    'featured_image_id': _get_filer_info,
    'is_sponsored': '',
    'sponsored_logo_id': _get_filer_info,
    'publication_date': '',
    'categories': _get_categories,
    'tags': _get_tags,
    'related_entries': _get_related_entries,
}

In [9]:
with open('entries.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=entry_fields.keys(), quoting=csv.QUOTE_MINIMAL, quotechar='"')
    writer.writeheader()
    for e in Entry.objects.all().prefetch_related():
        row = entry_fields.copy()
        for attr in entry_fields.keys():
            if row[attr] == '':
                row[attr] = getattr(e, attr)
                continue
            row[attr] = row[attr](e, attr)
        writer.writerow(row)